## De module `GeoPandas`

[GeoPandas](https://geopandas.org/en/stable/) is een open source-project om het werken met ruimtelijke gegevens in python gemakkelijker te maken. GeoPandas breidt de datatypes uit die door pandas worden gebruikt om ruimtelijke bewerkingen op geometrische types mogelijk te maken. Geometrische bewerkingen worden uitgevoerd door [shapely](https://shapely.readthedocs.io/en/stable/index.html). GeoPandas is verder afhankelijk van [fiona](https://pypi.org/project/Fiona/) voor bestandstoegang en [matplotlib](https://matplotlib.org/) voor plotten.

[Hier](https://geopandas.org/getting_started/introduction.html) vind je een uitgebreide GeoPandas introductie en voorbeeldscripts.

Om het GeoPandas te kunnen werken, beginnen we met het importeren van de noodzakelijke modules.

In [ ]:
# Import modules
# LET OP: 
# Installeer eerst Fiona en RTREE van https://www.lfd.uci.edu/~gohlke/pythonlibs/
# Dan geopandas installeren
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

# Set current dir as working dir

os.chdir(os.getcwd())
print('Working directory is ' + str(os.getcwd()))

In dit voorbeeld gaan we achtereenvolgens:
- Een csv-bestand *world_earthquakes.csv* met de locatie en magnitude van aardbevingen inlezen
- De aardbevingen plotten op een wereldkaart
- Via een spatial join alleen de aardbevingen selecteren die op land hebben plaatsgevonden
- Resultaat opslaan in Geojson file

We beginnen met het inlezen van het csv-bestand waarbij alle rijen die geen coördinaten en/of magnitude hebben (kolommen *latitude* en/of *longitude* en/of *eq_primary* zijn leeg) niet ingelezen worden.

In [ ]:
# Import world earthquales csv into dataframe and drop rows without latitude and longitude
df = pd.read_csv('data_files/world_earthquakes.csv',sep=',').dropna(subset=["LATITUDE", "LONGITUDE","EQ_PRIMARY"])

# Printing column names (when first row is not skipped and first 5 rows)
print(df.columns.values)
print(df[['COUNTRY', 'EQ_PRIMARY','LATITUDE', 'LONGITUDE']].head(5))

In de volgende stap maken we een geodataframe van het dataframe met het csv-bestand. Er wordt een extra kolom aan het dataframe toegoevoegd waarin de coördinaat van de aardbeving opgeslagen wordt. Deze coördinaat wordt gemaakt op basis van de x-waarde in de kolom *longitude* en de y-waarde in de kolom *latitude* met de functie [points_from_xy](https://geopandas.org/en/stable/docs/reference/api/geopandas.points_from_xy.html). Bij de parameter *crs* moet het coördinaatsysteem van de coördinaten opgegeven worden. In dit geval [WGS84](https://epsg.io/4326) met EPSG:4326.

In [ ]:
# Make geodata frame
gdf_earthquakes = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['LONGITUDE'],df['LATITUDE']),crs='EPSG:4326')

# Print first 5 geometries (as WKT)
print(gdf_earthquakes['geometry'].head(5))

Om te controleren of de aardbevingen goed ingeladen zijn, gaan we deze plotten op een wereldkaart. We gebruiken de dataset *naturalearth_lowres* als achtergrondkaart. De kaart wordt meegeleverd met de installatie van GeoPandas.

In [ ]:
# Import world map
print('Use ' + str(gpd.datasets.get_path("naturalearth_lowres")) + ' as world map')
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Set color and edgecolor
ax = world.plot(color='white', edgecolor='black')

# We can now plot our GeoDataFrame with earthquakes in red
gdf_earthquakes.plot(ax=ax, color='red')

# Show map
plt.show()

We willen gaan tellen hoeveel aardbeving op land hebben plaatsgevonden. We gaan dit doen door middel van een spatial query waarbij we alleen de aardbevingen selecteren die binnen een grenzen (polygoon) van een land vallen. We beginnen met het importeren van de dataset *UIA_World_Countries_Boundaries.shp* voor de polygonen van de landen en printen ter controle de kolommen en de eerste polygoon.

In [ ]:
# Import world shapefile
gdf_countries = gpd.read_file("data_files/UIA_World_Countries_Boundaries.shp")
print(gdf_countries.columns.values)
print(gdf_countries['geometry'].head(1))

Nu kunnen we de spatial join uitvoeren. Check hiervoor eerst of beide datasets dezelfde coördinaatsystemen hebben. Als dit niet het geval is moet je de functie [to_crs](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html) gebruiken om te zorgen dat beide geodataframes hetzelfde coördinaatsysteem hebben. 

Daarna voeren we de spatial join uit met de functie [sjoin](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html). We voeren een inner join uit zodat alleen de rijen overblijven met aardbevingen die binnen een landpolygoon vallen. Als laatste worden het totaal aantal aardbevingen in het geodataframe en het aantal aardbevingen op land geprint.

In [ ]:
# Check coördinate systems
print('Coordinate system earthquakes: ' + str(gdf_earthquakes.crs))
print('Coordinate system countries: ' + str(gdf_countries.crs))

# Spatial join
if gdf_earthquakes.crs == gdf_countries.crs:
    join_inner_df = gpd.sjoin(gdf_earthquakes, gdf_countries, how='inner')
else:
    print('Different coordinate systems, cannot perform spatial join')

# Print results
print('Total number of earthquakes: ' + str(len(gdf_earthquakes)))
print('Total number of earthquakes on land: ' + str(len(join_inner_df)))

We willen alleen de kolommen *COUNTRY*, *LOCATION_NAME*, *EQ_PRIMARY* en *geometry* opslaan in een Geojson. Ook de rijen zonder magnitude willen we niet meenemen.

In [ ]:
# Make new dataframe with 4 columns only (select columns)
gdf_new = join_inner_df[['COUNTRY', 'LOCATION_NAME', 'EQ_PRIMARY','geometry']]

# Select only rows with magnitude
gdf_cleaned = gdf_new[gdf_new['EQ_PRIMARY'].notna()]
print(gdf_cleaned.head(5))

# Export to geojson
gdf_new.to_file("data_files/world_earthquakes.json", driver="GeoJSON")